先前的模拟工作中，还有几部分遗留问题未处理：
1. G4模拟的质子能量输入的峰宽，中后两块Si统一设定为20keV。而在实验中，前后两块Si的厚度不一致，电容差异较大，分辨不同。同时峰宽会随着能量变大而变大。因而需要对输入的质子峰宽进行调整。
2. 研究β叠加效应时，未对DSSD进行分块处理。这一点会导致叠加谱可能会叠加多个DSSD小块的β能量，从而产生偏差。需要将DSSD进行分块，只叠加上同位置的DSSD小块沉积的β能量。
3. β发射谱的形状目前是较为对称的，缺少库伦扭曲因子的修正，这一点将会产生多大的效应，还需要好好进行研究。
4. G4模拟时采用的QBBC模型，不同模型下有何区别？调用不同版本的G4，将会有什么差异？

### 质子峰宽度调整

In [1]:
%jsroot on
double f1(double x)
{
    double energy[25] = {475, 721, 975, 1033, 1223, 1299, 1551, 1753, 2072, 2503, 2583, 2838, 3088, 3484, 4017, 4224, 4464, 4912, 5177, 5667, 5808, 5909, 6085, 6774, 7517};
    double br[25] = {4.73, 7.39, 0.25, 3, 0.75, 18.51, 0.81, 0.45, 0.48, 0.64, 4.89, 2.11, 1.89, 2.18, 1.078, 0.84, 3.21, 0.27, 0.29, 0.35, 0.18, 0.21, 0.41, 0.41, 0.33};
    double s1[25] = {17.91,19,20,17.77,20,21.25,20,20,20,25.5,20,20,20,36.42,20,37.21,20,20,20,20,20,20,20,20,20};
    //double s2[25] = {16.12,15.89,20,15.23,20,15.68,20,20,20,21.13,20,20,20,37.01,20,39.28,20,20,20,20,20,20,20,20,20};
   // double sig = 20;
    double c[25];
    double sum = 0;
    for (int i = 0; i < 25; i++)
    {
        c[i] = br[i] * TMath::Gaus(x, energy[i], s1[i]);
        sum += c[i];
    }
    return sum;
}


In [2]:
double f2(double x)
{
    double energy[25] = {475, 721, 975, 1033, 1223, 1299, 1551, 1753, 2072, 2503, 2583, 2838, 3088, 3484, 4017, 4224, 4464, 4912, 5177, 5667, 5808, 5909, 6085, 6774, 7517};
    double br[25] = {4.73, 7.39, 0.25, 3, 0.75, 18.51, 0.81, 0.45, 0.48, 0.64, 4.89, 2.11, 1.89, 2.18, 1.078, 0.84, 3.21, 0.27, 0.29, 0.35, 0.18, 0.21, 0.41, 0.41, 0.33};
    //double s1[25] = {17.91,19,20,17.77,20,21.25,20,20,20,25.5,20,20,20,36.42,20,37.21,20,20,20,20,20,20,20,20,20};
    double s2[25] = {16.12,15.89,20,15.23,20,15.68,20,20,20,21.13,20,20,20,37.01,20,39.28,20,20,20,20,20,20,20,20,20};
   // double sig = 20;
    double c[25];
    double sum = 0;
    for (int i = 0; i < 25; i++)
    {
        c[i] = br[i] * TMath::Gaus(x, energy[i], s2[i]);
        sum += c[i];
    }
    return sum;
}

In [3]:
void dis()
{
    TF1 *fa1 = new TF1("fa1", "f1(x)", 0, 8000);
    fa1->SetNpx(8000);
    TF1 *fa2 = new TF1("fa2","f2(x)",0,8000);
    fa2->SetNpx(8000);
    TFile *opf = new TFile("edis.root", "recreate");
    TH1F *h1 = new TH1F("energydis1", "energydis1", 8000, 0, 8000);
    h1->FillRandom("fa1", 10000000);
    h1->Write();
    TH1F *h2 = new TH1F("energydis2", "energydis2", 8000, 0, 8000);
    h2->FillRandom("fa2", 10000000);
    h2->Write();
    opf->Close();
}
dis();

In [4]:
TCanvas c1;
TFile *ipf=new TFile("edis.root");
energydis1->Draw();
energydis2->SetLineColor(2);
energydis2->Draw("same");
c1.Draw();

蓝色的线为中间Si的质子输入谱，红色的线为后Si的质子输入谱。

In [5]:
TFile *s1=new TFile("simu0720.root");
tree->Draw("de[1][0]>>p1_010(2000,0,8)","de[2][0]<0.01&&de[0][0]<0.01&&de[1][0]>0");
tree->Draw("de[1][1]>>b1_010(2000,0,8)","de[2][1]<0.01&&de[0][1]<0.01&&de[1][1]>0");
tree->Draw("de[1][2]>>b1_020(2000,0,8)","de[2][2]<0.01&&de[0][2]<0.01&&de[1][2]>0");
tree->Draw("de[1][2]-de[1][1]>>b1dif(2000,0,8)","de[2][1]<0.01&&de[0][1]<0.01&&de[1][1]>0&&de[1][2]>de[1][1]");
tree->Draw("de[1][0]+de[1][1]>>bp1_010(2000,0,8)","de[2][0]<0.01&&de[0][0]<0.01&&de[1][0]>0");
tree->Draw("de[1][0]+de[1][1]>>brp1_010(2000,0,8)","de[2][0]<0.01&&de[0][0]<0.01&&de[1][0]>0&&vetz[1][1]>0");
tree->Draw("de[1][0]+de[1][1]>>blp1_010(2000,0,8)","de[2][0]<0.01&&de[0][0]<0.01&&de[1][0]>0&&vetz[1][1]<=0");

tree->Draw("de[1][0]+de[1][2]>>bp1_020(2000,0,8)","de[2][0]<0.01&&de[0][0]<0.01&&de[1][0]>0");
tree->Draw("de[1][0]+de[1][2]>>brp1_020(2000,0,8)","de[2][0]<0.01&&de[0][0]<0.01&&de[1][0]>0&&vetz[1][2]>0");
tree->Draw("de[1][0]+de[1][2]>>blp1_020(2000,0,8)","de[2][0]<0.01&&de[0][0]<0.01&&de[1][0]>0&&vetz[1][2]<=0");

tree->Draw("de[1][0]>>pr1_010(2000,0,8)","de[2][0]<0.01&&de[0][0]<0.01&&de[1][0]>0&&vetz[1][0]>0");
tree->Draw("de[1][0]>>pl1_010(2000,0,8)","de[2][0]<0.01&&de[0][0]<0.01&&de[1][0]>0&&vetz[1][0]<=0");
tree->Draw("de[1][1]>>br1_010(2000,0,8)","de[2][1]<0.01&&de[0][1]<0.01&&de[1][1]>0&&vetz[1][1]>0");
tree->Draw("de[1][1]>>bl1_010(2000,0,8)","de[2][1]<0.01&&de[0][1]<0.01&&de[1][1]>0&&vetz[1][1]<=0");

In [6]:
c1.Clear();
c1.Divide(1,2);
c1.cd(1);
p1_010->SetLineColor(1);
pr1_010->SetLineColor(2);
pl1_010->SetLineColor(3);
p1_010->Draw();
pr1_010->Draw("same");
pl1_010->Draw("same");
c1.cd(2);
b1_010->SetLineColor(1);
br1_010->SetLineColor(2);
bl1_010->SetLineColor(3);
b1_010->Draw();
br1_010->Draw("same");
bl1_010->Draw("same");
c1.Draw();

上图为质子谱，下图为beta谱。黑线为总谱，红线为向右发射，绿线为向左发射。对于电子而言，体现为出现两个峰。向左逃逸的β产生了14keV的偏移，向左偏移的电子使得质子峰变胖。

In [7]:
c1.Clear();
c1.Divide(2,1);
c1.cd(1);
TF1 *fitexp=new TF1("fitexp","expo",0,2);
TF1 *fitlandau=new TF1("fitlandau","landau",0,3);
fitexp->SetNpx(8000);
fitexp->SetParameters(11.44,-96.23);
fitlandau->SetNpx(8000);
fitlandau->SetParameters(5.327e5,0.0138,0.003731);
br1_010->Fit(fitexp,"R");
c1.cd(2);
bl1_010->Fit(fitlandau,"R");
c1.Draw();

 FCN=33845.2 FROM MIGRAD    STATUS=CONVERGED     163 CALLS         164 TOTAL
                     EDM=1.01779e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.14356e+01   3.63871e-03   1.83751e-04   1.32689e-02
   2  Slope       -9.62305e+01   2.85880e-01   1.44385e-02   1.71501e-03
 FCN=22096.7 FROM MIGRAD    STATUS=CONVERGED     280 CALLS         281 TOTAL
                     EDM=1.43929e-12    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   1.2 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     5.26801e+05   1.19802e+03   8.46531e-01  -2.15605e-09
   2  MPV          1.39038e-02   1.13253e-05   2.57410e-09  -1.55715e-01
   3  Sigma        3.83653e-03   6.85509e-06  -1.56974e-08  -3.20116e-03


因而如果需要对质子峰进行拟合时，向右逃逸的部分需要用指数卷积高斯拟合，向左逃逸的部分需要使用朗道卷积高斯拟合。
对于实验谱，应当使用两个卷积加和进行拟合。

root提供的卷积示例代码
- [朗道高斯卷积](https://nbviewer.jupyter.org/url/root.cern.ch/doc/master/notebooks/langaus.C.nbconvert.ipynb)
- [指数高斯卷积](https://nbviewer.jupyter.org/url/root.cern.ch/doc/master/notebooks/fitConvolution.C.nbconvert.ipynb)

In [8]:
c1.Clear();
c1.Divide(2,1);
c1.cd(1);
b1_010->SetLineColor(1);
b1_020->SetLineColor(2);
b1_010->Draw();
b1_020->Draw("same");
c1.cd(2);
b1dif->Draw();
c1.Draw();

黑线只考虑一小块Si沉积能量，红线表示整块DSSD沉积的能量。两者并没有显著差异。

In [9]:
c1.Clear();
c1.Divide(1,2);
c1.cd(1);
bp1_010->SetLineColor(1);
brp1_010->SetLineColor(2);
blp1_010->SetLineColor(3);
p1_010->Draw();
bp1_010->Draw("same");
brp1_010->Draw("same");
blp1_010->Draw("same");
c1.cd(2);
bp1_020->SetLineColor(1);
brp1_020->SetLineColor(2);
blp1_020->SetLineColor(3);
p1_010->Draw();
bp1_020->Draw("same");
brp1_020->Draw("same");
blp1_020->Draw("same");
c1.Draw();

比较奇怪的是，往左逃逸的电子的质子叠加谱与质子谱峰位对齐，往右逃逸的偏移10keV左右。

分析后一块SI的谱


In [10]:
tree->Draw("de[2][0]>>p2_001(2000,0,8)","de[1][0]<0.01&&de[0][0]<0.01&&de[2][0]>0");
tree->Draw("de[2][1]>>b2_001(2000,0,8)","de[1][1]<0.01&&de[0][1]<0.01&&de[2][1]>0");
tree->Draw("de[2][2]>>b2_002(2000,0,8)","de[1][2]<0.01&&de[0][2]<0.01&&de[2][2]>0");
tree->Draw("de[2][2]-de[2][1]>>b2dif(2000,0,8)","de[1][1]<0.01&&de[0][1]<0.01&&de[2][1]>0&&de[2][2]>de[2][1]");

tree->Draw("de[2][0]>>pr2_001(2000,0,8)","de[1][0]<0.01&&de[0][0]<0.01&&de[2][0]>0&&vetz[2][0]>0");
tree->Draw("de[2][0]>>pl2_001(2000,0,8)","de[1][0]<0.01&&de[0][0]<0.01&&de[2][0]>0&&vetz[2][0]<=0");
tree->Draw("de[2][1]>>br2_001(2000,0,8)","de[1][1]<0.01&&de[0][1]<0.01&&de[2][1]>0&&vetz[2][1]>0");
tree->Draw("de[2][1]>>bl2_001(2000,0,8)","de[1][1]<0.01&&de[0][1]<0.01&&de[2][1]>0&&vetz[2][1]<=0");

tree->Draw("de[2][0]+de[2][1]>>bp2_001(2000,0,8)","de[1][0]<0.01&&de[0][0]<0.01&&de[2][0]>0");
tree->Draw("de[2][0]+de[2][1]>>brp2_001(2000,0,8)","de[1][0]<0.01&&de[0][0]<0.01&&de[2][0]>0&&vetz[2][1]>0");
tree->Draw("de[2][0]+de[2][1]>>blp2_001(2000,0,8)","de[1][0]<0.01&&de[0][0]<0.01&&de[2][0]>0&&vetz[2][1]<=0");

tree->Draw("de[2][0]+de[2][2]>>bp2_002(2000,0,8)","de[1][0]<0.01&&de[0][0]<0.01&&de[2][0]>0");
tree->Draw("de[2][0]+de[2][2]>>brp2_002(2000,0,8)","de[1][0]<0.01&&de[0][0]<0.01&&de[2][0]>0&&vetz[2][2]>0");
tree->Draw("de[2][0]+de[2][2]>>blp2_002(2000,0,8)","de[1][0]<0.01&&de[0][0]<0.01&&de[2][0]>0&&vetz[2][2]<=0");

In [11]:
c1.Clear();
c1.Divide(1,2);
c1.cd(1);
p2_001->SetLineColor(1);
pr2_001->SetLineColor(2);
pl2_001->SetLineColor(3);
p2_001->Draw();
pr2_001->Draw("same");
pl2_001->Draw("same");
c1.cd(2);
b2_001->SetLineColor(1);
br2_001->SetLineColor(2);
bl2_001->SetLineColor(3);
b2_001->Draw();
br2_001->Draw("same");
bl2_001->Draw("same");
c1.Draw();

In [12]:
c1.Clear();
c1.Divide(2,1);
c1.cd(1);
fitexp->SetParameters(12.78,-84);
fitlandau->SetParameters(5.14e5,0.1069,0.018);
br2_001->Fit(fitlandau,"R");
c1.cd(2);
bl2_001->Fit(fitexp,"R");
c1.Draw();

 FCN=565942 FROM MIGRAD    STATUS=CONVERGED     288 CALLS         289 TOTAL
                     EDM=1.65478e-09    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   0.6 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     5.13680e+05   5.22663e+02  -1.23218e-01   1.15072e-07
   2  MPV          1.06871e-01   2.82966e-05   1.02874e-08  -1.74921e+00
   3  Sigma        1.79813e-02   1.40471e-05   7.73412e-08   1.64654e+00
 FCN=258272 FROM MIGRAD    STATUS=CONVERGED     152 CALLS         153 TOTAL
                     EDM=7.62889e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.27776e+01   2.11222e-03   2.41577e-04  -2.11946e-01
   2  Slope       -8.35249e+01   1.54997e-01   1.77243e-02   5.29414e-03


向右逃逸的峰位偏移100keV。

In [13]:
c1.Clear();
c1.Divide(2,1);
c1.cd(1);
b2_001->SetLineColor(1);
b2_002->SetLineColor(2);
b2_001->Draw();
b2_002->Draw("same");
c1.cd(2);
b2dif->Draw();
c1.Draw();

In [14]:
c1.Clear();
c1.Divide(1,2);
c1.cd(1);
bp2_001->SetLineColor(1);
brp2_001->SetLineColor(2);
blp2_001->SetLineColor(3);
p2_001->Draw();
bp2_001->Draw("same");
brp2_001->Draw("same");
blp2_001->Draw("same");
c1.cd(2);
bp2_002->SetLineColor(1);
brp2_002->SetLineColor(2);
blp2_002->SetLineColor(3);
p2_001->Draw();
bp2_002->Draw("same");
brp2_002->Draw("same");
blp2_002->Draw("same");
c1.Draw();

最高峰的计数对应的为纯质子峰，黑线为β叠加谱。绿线对β向左逃逸的β叠加谱，红线为β向右逃逸的β叠加谱。红线的锋味右移了100keV。由于之前分析过不同Q值下沉积的β能一致，因而不同峰偏移的程度应该一致。

In [21]:
c1.Clear();
c1.Divide(1,2);
s1->cd();
c1.cd(1);
bp1_010->Draw();
//TFile *s2=new TFile("al.root");
s2->cd();
c1.cd(2);
al1->Draw();
c1.Draw();

In [26]:
c1.Clear();
c1.Divide(1,2);
s1->cd();
c1.cd(1);
p2_001->Draw();
//TFile *s2=new TFile("al.root");
s2->cd();
c1.cd(2);
al2->Draw();
c1.Draw();

In [29]:
c1.Clear();
c1.Divide(1,2);
s1->cd();
c1.cd(1);
bp2_001->Draw();
//TFile *s2=new TFile("al.root");
s2->cd();
c1.cd(2);
al2->Draw();
c1.Draw();

In [31]:
c1.Clear();
c1.Divide(1,2);
s1->cd();
c1.cd(1);
brp2_001->Draw();
//TFile *s2=new TFile("al.root");
s2->cd();
c1.cd(2);
al2->Draw();
c1.Draw();

In [32]:
c1.Clear();
c1.Divide(1,2);
s1->cd();
c1.cd(1);
blp2_001->Draw();
//TFile *s2=new TFile("al.root");
s2->cd();
c1.cd(2);
al2->Draw();
c1.Draw();

上面给出的分别为不同条件下的质子谱。各有类似的地方。

分别拟合质子峰和叠加β后的质子峰，叠加β后质子峰明显变宽。即β叠加后质子峰变胖，因而输入的β谱分辨应相应地调低。